**This example of code is the approximator for Gauchonator 1.0.**

In this case, instead of using a linear approach, a neural network is used. As you can see, we are in the training process, 48x of loss is too much.

In [1]:
%tensorflow_version 2.x
from __future__ import absolute_import, division, print_function, unicode_literals
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
from tensorflow.keras import Sequential
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import ast

try:
  import tensorflow.compat.v2 as tf
except Exception:
  pass

dir_base_model = "/content/gdrive/My Drive/DeepLearning/2d-mapper/mapper-2dt"
tf.enable_v2_behavior()
print(tf.__version__)

TensorFlow 2.x selected.
2.1.0-rc1


In [2]:
# Load input data
from google.colab import drive
drive.mount('/content/gdrive')
data = pd.read_csv(dir_base_model + ".csv", index_col=False)#("X","Y","S","T"))

v = int(len(data) * 0.8)
train_data = data[1:v]
test_data = data[v:len(data)]

x_train = train_data[["S", "T"]]
y_train = train_data[["X", "Y"]]
x_test = test_data[["S", "T"]]
y_test = test_data[["X", "Y"]]

x_train.columns = [''] * len(x_train.columns)
y_train.columns = [''] * len(y_train.columns)

# Print train set
#print(x_train)
#train_data[["S", "T"]]

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import datetime

class MyCustomCallback(tf.keras.callbacks.Callback):
  def __init__():
    self.counterTest = 0
    self.counterTrain = 0
  
  def on_train_batch_end(self, batch, logs=None):
    print('Para el batch {}, la perdida (loss) es {:7.2f}.'.format(batch, logs['loss']))

  def on_test_batch_end(self, batch, logs=None):
    print('Para el  batch {}, la perdida (loss) es {:7.2f}.'.format(batch, logs['loss']))

  def on_epoch_end(self, epoch, logs=None):
    if (self.counterTest % 100):
      print('La perdida promedio para la epoch {} es {:7.2f} y el MAE es {:7.2f}.'.format(epoch, logs['loss'], logs['mae']))
    self.counterTest = self.counterTest + 1

  #def on_train_batch_begin(self, batch, logs=None):
  #  if (self.counterTest % 100):
  #    pass
  #    #print('Entrenamiento: batch {} comienza en {}'.format(batch, datetime.datetime.now().time()))

  #def on_train_batch_end(self, batch, logs=None):
  #  if (self.counterTest % 100):        
  #    print('Entrenamiento: batch {} termina en {}'.format(batch, datetime.datetime.now().time()))
  #  self.counterTrain = self.counterTrain + 1

  #def on_test_batch_begin(self, batch, logs=None):
  #  if (self.counterTest % 100):
  #    pass
  #    #print('Evaluacion: batch {} comienza en {}'.format(batch, datetime.datetime.now().time()))

  #def on_test_batch_end(self, batch, logs=None):
  #  if (self.counterTest % 100):
  #    print('Evaluacion: batch {} termina en {}'.format(batch, datetime.datetime.now().time()))
  #  self.counterTest = self.counterTest + 1

In [35]:
# Esta es la mejor forma que encontre, pero seguramente colgando un poco entrenando se pueda mejorar MUCHO.
# Lindo caso para analizar bien.
# 
# Build model
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(2, activation='linear'),
  tf.keras.layers.Dense(12, activation='linear'),
  tf.keras.layers.Dense(12, activation='linear'),
  tf.keras.layers.Dense(2, activation='linear'),
])

#optimizer = tf.keras.optimizers.RMSprop(0.001)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.02)

model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])
#model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.fit(x = x_train.values, y = y_train.values, verbose=0, epochs=1000)
#model.fit(x = x_train.values, y = y_train.values, verbose=1, epochs=200, callbacks=[MyCustomCallback])
#model.fit(x = x_train.values, y = y_train.values, validation_split=0.1, batch_size = 256, verbose=2, epochs=5)
print(model.summary())

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_35 (Dense)             multiple                  6         
_________________________________________________________________
dense_36 (Dense)             multiple                  36        
_________________________________________________________________
dense_37 (Dense)             multiple                  156       
_________________________________________________________________
dense_38 (Dense)             multiple                  26        
Total params: 224
Trainable params: 224
Non-trainable params: 0
_________________________________________________________________
None


In [38]:
try:
    f = open(dir_base_model + '-info.txt', 'rt')
    f.close()
    exists = True
except FileNotFoundError:
    exists = False

# Check if saved model is better. If false, save this one.
if (exists == True):
  file = open(dir_base_model + '-info.txt', 'rt')
  value = file.read()
  file.close()
  old_evaluation_result = ast.literal_eval(value)
  print("Saved loss is: " + str(old_evaluation_result))

# Evaluate model
evaluation_result = model.evaluate(x_test.values,  y_test.values, verbose=2)
print("Current loss is: " + str(evaluation_result))

# Save only if found a better configuration
if (exists == False or evaluation_result[0] < old_evaluation_result[0]):
  print("Found better model with loss: " + str(evaluation_result))
  file = open(dir_base_model + '-info.txt', 'wt')
  file.write(str(evaluation_result))
  file.close()
  model.save(dir_base_model + '.model')
else:
  print("Existing is ok.\n")

# Get weights
#print("Current weights:\n")
#print(model.get_weights())

Saved loss is: [10.60622501373291, 2.8289435, 10.606225]
7/7 - 0s - loss: 10.6489 - mae: 2.8354 - mse: 10.6489
Current loss is: [10.648908615112305, 2.8353713, 10.648909]
Existing is ok.



In [24]:
# Prediccion
predictions = model.predict(x_test[:3].values)
print(x_test[:3])
print(y_test[:3])
print("Predicted: ")
print(predictions[:3])

        S      T
28  497.0   85.0
29  499.0   16.0
30  563.0  468.0
       X     Y
28 -10.0  20.0
29 -10.0  30.0
30   0.0 -20.0
Predicted: 
[[-15.186491   28.178219 ]
 [-13.244334   34.63084  ]
 [-25.746077   -3.0341187]]


**Let's see training data:**

In [7]:
import plotly.express as px
fig = px.line(data, x="X", y="S", labels={'X':'S'})
fig.show()
fig = px.line(data, x="Y", y="T", labels={'Y':'T'})
fig.show()

In [8]:
import plotly.express as px
fig = px.scatter(data, x="X", y="S", title="X axis translation")
fig.show()
fig = px.scatter(data, x="Y", y="T", title="Y axis translation")
fig.show()
fig = px.scatter(data, x="S", y="T", title="Point map translation (show slight deformation)")
fig.show()